In [17]:
%config IPCompleter.greedy=True

import import_ipynb
from functions import get_upcoming_team_fixtures_data
from functions import get_upcoming_fixtures_data

import os
import requests
import pandas

In [99]:
import plotly.offline
import plotly.graph_objects as go

import chart_studio
import chart_studio.plotly as py
#chart_studio .tools.set_credentials_file(username='user', api_key='***')

In [129]:
def next_fixtures(no_fixtures=6, limit_diff=10, no_above_limit_occ=20):
    clubs = get_upcoming_fixtures_data(base_path, season)['team_a'].unique()
    fig = go.Figure()

    for club in clubs:
        df = get_upcoming_team_fixtures_data(club, base_path, season).head(no_fixtures)
        if (max(df['difficulty'])>limit_diff & len(df[df['difficulty'] > limit_diff]) > no_above_limit_occ):
            continue
            
        fig.add_trace(go.Scatter(
            x=df['event'], 
            y=df['difficulty'],
            name=club,
            text = df['opponent'],
            hoverlabel= dict(
                font=dict(color='#404040'),
                bordercolor='#404040',
                bgcolor='white'
            ),
            hovertemplate = "<b>"+club+"</b></br></br>vs %{text}</br></br><extra></extra>"))
        
    fig.update_layout(
        legend=go.layout.Legend(
            traceorder="normal",
            font=dict(color="#eee"),
            bgcolor="rgba(0,0,0,0)"
        )
    )
    
    fig.update_yaxes(title_text="Difficulty",color='#eee')
    fig.update_xaxes(title_text="Gameweek",color='#eee')

    fig.update_scenes(bgcolor='rgba(0,0,0,0)')

    fig.layout.update(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    chart_studio.plotly.plot(fig, filename="easiest_schedule")
#    plotly.offline.iplot(fig)

In [130]:
season='2019-20'
base_path = 'C:/Users/antonia/source/antoniaelek/fantasy-premier-league/scraper/'

In [131]:
next_fixtures(limit_diff=3, no_above_limit_occ=1)